# Building Agents with LangGraph & Llama Stack

In the last notebook, you built a ReAct agent **from scratch**. You wrote:
- Manual response parsing with regex
- Custom iteration loops
- State management
- Tool execution logic
- Error handling

That was ~200+ lines of code. Now let's see how **LangGraph** makes this dramatically simpler.

## What We'll Build

A knowledge-based assistant that:
1. Searches documents for answers (RAG)
2. Schedules meetings with professors if it can't answer

Same capabilities. Way less code.

## Setup: Install Dependencies

In [ ]:
!pip3 install -q langgraph==0.2.62 langchain-openai==0.3.7 langchain-core==0.3.28 llama-stack-client==0.3.0

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from llama_stack_client import LlamaStackClient
import json

print("✅ All imports successful")

## Connect to Llama Stack

LangGraph connects to Llama Stack through **OpenAI-compatible endpoints**. This means we can use LangChain's `ChatOpenAI` client!

In [ ]:
# LangGraph connects via OpenAI-compatible endpoint
llm = ChatOpenAI(
    base_url="http://llama-stack-service:8321/v1/openai/v1",
    model="llama32",
    api_key="not-needed",  # Llama Stack doesn't require auth in this setup
    temperature=0.1
)

# Also connect standard client for RAG
llama_client = LlamaStackClient(base_url="http://llama-stack-service:8321")

print("✅ Connected to Llama Stack")
print(f"   OpenAI-compatible endpoint: http://llama-stack-service:8321/v1/openai/v1")
print(f"   Model: llama32")

## Define Tools

With LangGraph, tools are just Python functions with the `@tool` decorator. No manual JSON schemas!

In [ ]:
# First, let's get our vector store for RAG
try:
    vector_stores = llama_client.vector_stores.list()
    
    if vector_stores.data and len(vector_stores.data) > 0:
        vector_store = vector_stores.data[0]
        print(f"✅ Using vector store: {vector_store.id}")
    else:
        vector_store = llama_client.vector_stores.create(
            name="langgraph_knowledge_base",
            extra_body={
                "embedding_model": "all-MiniLM-L6-v2",
                "embedding_dimension": 384,
                "provider_id": "milvus",
                "vector_db_id": "langgraph_kb"
            }
        )
        print(f"✅ Created vector store: {vector_store.id}")
        print("⚠️  Note: Ingest documents first (see 5-rag notebooks)")
except Exception as e:
    print(f"⚠️  Vector store setup failed: {e}")
    vector_store = None

In [ ]:
@tool
def search_knowledge_base(query: str) -> str:
    """Search through documents to find information. Use this when the user asks about concepts, definitions, or topics."""
    if not vector_store:
        return "Error: Knowledge base not available. Please set up the vector store first."
    
    try:
        results = llama_client.vector_stores.search(
            vector_store_id=vector_store.id,
            query=query,
            max_num_results=3,
            search_mode="vector"
        )
        
        if not results.data:
            return "No relevant information found in the knowledge base."
        
        formatted_results = []
        for i, result in enumerate(results.data, 1):
            content = result.content if hasattr(result, 'content') else str(result)
            formatted_results.append(f"Result {i}: {content}")
        
        return "\n\n".join(formatted_results)
    except Exception as e:
        return f"Error searching knowledge base: {str(e)}"


@tool
def schedule_professor_meeting(professor_name: str, topic: str, proposed_time: str) -> str:
    """Schedule a meeting with a professor. Use this when you can't answer a question and need expert help.
    
    Args:
        professor_name: Name of the professor to meet with
        topic: What the meeting is about
        proposed_time: Suggested meeting time (e.g., 'tomorrow at 2pm', 'next Monday morning')
    """
    # In a real system, this would integrate with a calendar API
    return f"✅ Meeting scheduled with {professor_name} for '{topic}' at {proposed_time}. You'll receive a calendar invite shortly."


# List of available tools
tools = [search_knowledge_base, schedule_professor_meeting]

print("✅ Tools defined:")
for tool_func in tools:
    print(f"   - {tool_func.name}: {tool_func.description[:60]}...")

## Build the Agent

This is where the magic happens. With LangGraph, creating a ReAct agent is **one line of code**.

No parsing. No loops. No state management. LangGraph handles it all.

In [ ]:
# Create the ReAct agent - that's it!
agent = create_react_agent(
    llm,
    tools,
    checkpointer=MemorySaver()  # Enables conversation memory
)

print("✅ ReAct agent created!")
print("\n🎯 Compare this to the 200+ lines you wrote before...")

### What Just Happened?

That one function call replaces ALL of this from your previous notebook:
- ❌ `parse_react_response()` - Manual regex parsing
- ❌ `execute_tool()` - Tool routing logic
- ❌ `run_react_agent()` - The entire iteration loop
- ❌ Conversation history management
- ❌ Error handling and retry logic
- ❌ Stopping condition checks

LangGraph does **all of this automatically**.

## Test the Agent

Let's see it in action!

In [ ]:
def run_agent(question: str, thread_id: str = "default"):
    """Run the agent with a question and display the conversation."""
    print("="*70)
    print(f"QUESTION: {question}")
    print("="*70)
    
    config = {"configurable": {"thread_id": thread_id}}
    
    inputs = {
        "messages": [
            SystemMessage(content="You are a helpful assistant. When you can't answer a question from the knowledge base, offer to schedule a meeting with the relevant professor."),
            HumanMessage(content=question)
        ]
    }
    
    # Stream the agent's work
    for event in agent.stream(inputs, config, stream_mode="values"):
        last_message = event["messages"][-1]
        
        # Display the message
        if hasattr(last_message, 'content') and last_message.content:
            print(f"\n{last_message.type.upper()}: {last_message.content}")
        
        # Display tool calls
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            for tool_call in last_message.tool_calls:
                print(f"\n🔧 CALLING TOOL: {tool_call['name']}")
                print(f"   Args: {json.dumps(tool_call['args'], indent=2)}")
    
    print("\n" + "="*70)

print("✅ Helper function ready")

### Example 1: Knowledge Base Search

The agent should search the knowledge base for this information.

**Note**: This requires documents in your vector store. If you haven't done the RAG notebooks (`5-rag/2-intro-to-RAG.ipynb`), the knowledge base will be empty.

In [ ]:
run_agent("What is the structure of a forest canopy in botany?")

### Example 2: Scheduling a Meeting

When the agent can't answer a question, it should offer to schedule a meeting with an expert.

In [ ]:
run_agent("I need help with quantum chromodynamics calculations. Can you help?")

### Example 3: Multi-Step Reasoning

Watch the agent autonomously decide between searching and scheduling.

In [ ]:
run_agent("What are agentic workflows, and can I meet with someone to learn more about implementing them?")

## The Comparison

| **From Scratch** | **With LangGraph** |
|------------------|--------------------|
| 200+ lines of code | ~50 lines of code |
| Manual response parsing with regex | Automatic parsing |
| Custom iteration loop | Built-in ReAct loop |
| Manual state management | Automatic state handling |
| Custom tool routing | Automatic tool selection |
| Manual conversation history | Built-in memory |
| Custom error handling | Built-in error recovery |
| Hard to extend | Easy to add new tools |

### Key Benefits of LangGraph

1. **Declarative**: You define *what* the agent should do, not *how*
2. **Production-Ready**: Built-in error handling, retries, and state management
3. **Extensible**: Adding new tools is trivial (just add a decorated function)
4. **Observable**: Built-in streaming and logging
5. **Maintainable**: Much less code to debug and maintain

## Try It Yourself!

Now experiment:
1. Ask questions that require knowledge base searches
2. Ask questions that should trigger meeting scheduling
3. Try to make the agent use both tools in sequence

See how the agent autonomously reasons about which tool(s) to use!

In [ ]:
# Your turn!
run_agent("YOUR QUESTION HERE")

## Bonus: Adding a New Tool

Remember how much work it was to add a new tool to your from-scratch agent?

With LangGraph, it's this easy:

In [ ]:
@tool
def get_course_schedule(course_code: str) -> str:
    """Get the schedule for a specific course."""
    # Mock data - in reality this would query a course database
    schedules = {
        "CS101": "Mondays and Wednesdays, 10:00 AM - 11:30 AM",
        "CS201": "Tuesdays and Thursdays, 2:00 PM - 3:30 PM",
        "MATH301": "Mondays, Wednesdays, Fridays, 9:00 AM - 10:00 AM"
    }
    return schedules.get(course_code, f"No schedule found for {course_code}")

# Add it to the tools list
tools.append(get_course_schedule)

# Recreate the agent with the new tool
agent = create_react_agent(llm, tools, checkpointer=MemorySaver())

print("✅ New tool added!")
print("\nThat's it. No changes to parsing logic, no changes to the loop, no changes to tool routing.")
print("The agent just... works.")

In [ ]:
# Test the new tool
run_agent("When is CS101 scheduled?")

## Key Takeaways

1. **Frameworks save massive amounts of time**: 200+ lines → 50 lines
2. **Frameworks are production-ready**: Built-in error handling, state management, and observability
3. **Frameworks are flexible**: LangGraph works with any OpenAI-compatible endpoint (including Llama Stack)
4. **Understanding the fundamentals still matters**: Knowing *how* ReAct works helps you debug when things go wrong

## Next Steps

In production environments:
- Add authentication and authorization
- Implement proper error handling and monitoring
- Add logging and tracing (LangGraph integrates with LangSmith)
- Scale with distributed execution
- Implement rate limiting and cost controls

All of these are easier with a mature framework like LangGraph!